In [21]:
from my_tokenizers2 import SmilesTokenizer
import numpy as np
import torch
tokenizer = SmilesTokenizer(vocab_file ='mol2vec_vocabs.txt')


Setting 'max_len_single_sentence' is now deprecated. This value is automatically set up.
Setting 'max_len_sentences_pair' is now deprecated. This value is automatically set up.


In [22]:
from transformers import ElectraModel, ElectraConfig,ElectraForPreTraining,ElectraForMaskedLM
emb_dim = 256
output_dir="./fingerprints_smile_output256/full"
electra_model = ElectraForMaskedLM.from_pretrained(output_dir)

In [23]:
import numpy as np
import csv 
trfile = open('../original-dataset/regression/Cep-Duvenaud.csv', 'r')
reader = csv.reader(trfile)
next(reader)
smiles,dataY,all_smiles = [], [], []
for raw in reader:
    # print(raw)
    dataY.append(float(raw[-1]))
    smiles = str(raw[-2])
    all_smiles.append(smiles)


    

In [24]:
#before removing duplications
print(len(all_smiles))
print(len(dataY))

29978
29978


In [25]:
from collections import Counter, defaultdict
def duplicates(lst):
    cnt= Counter(lst)
    return [key for key in cnt.keys() if cnt[key]> 1]
dups = duplicates(all_smiles)


In [26]:
print(len(dups))

0


In [27]:
newY = []
newSm = []

"""
for i,x in enumerate(all_smiles):
    if x not in dups:
        newY.append(dataY[i])
        newSm.append(x)
"""
for i,x in enumerate(all_smiles):
    newY.append(dataY[i])
    newSm.append(x)

In [28]:
#after removing duplications
print(len(newSm))
print(len(newY))

29978
29978


In [29]:
from tqdm import tqdm
from rdkit import Chem
from mol2vec.features import mol2alt_sentence

#smiles_seqs_emb = {}
#smiles_aug_dict = {}

embX = []
MAX_SMI_LEN = 256

for item in tqdm(newSm):
    
    smiles = item
    aas = Chem.MolFromSmiles(item) 
    sentence = mol2alt_sentence(aas, 1)
    sentence = ' '.join(map(str, sentence)) 
    # 替换
#     if len(item) > MAX_SMI_LEN:
#         input_ids = tokenizer(sentence[:MAX_SMI_LEN], return_tensors="pt")["input_ids"]
#     else:
#         input_ids = tokenizer(sentence, return_tensors="pt")["input_ids"]
    input_ids = tokenizer(sentence, return_tensors="pt")["input_ids"]
    if len(input_ids[0]) > MAX_SMI_LEN:
        print(smiles)
        input_ids = input_ids[:, :MAX_SMI_LEN]
        input_ids[:,255:256] = 3

    outputs = electra_model(input_ids, labels=input_ids)
    #smiles_seqs_emb[smiles] = outputs[2][0].detach().numpy()[0][1:-1]
    embX.append(outputs[2][0].detach().numpy()[0][1:-1])
#         if item not in smiles_aug_dict.keys():
#             smiles_aug_dict[item] = smiles
#         else:
#             smiles_aug_dict[item] =smiles_aug_dict[item] +';'+ smiles
#embX[0].shape            
        

        
        

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29978/29978 [24:40<00:00, 20.25it/s]


In [31]:
np.save('../encoded-dataset/regression/Cep_embX_fullBERT', embX)
np.save('../encoded-dataset/regression/Cep_Y_fullBERT', newY)